In [9]:
import requests
from bs4 import BeautifulSoup

In [10]:
url = 'https://www.rakuten-sec.co.jp/web/market/data/list.html'
res = requests.get(url)
soup = BeautifulSoup(res.content, "html.parser")

In [11]:
#elems = soup.find_all('tr')
elems = soup.find_all('th')
elems

[<th class="w-30" scope="col">指標</th>,
 <th class="align-C" scope="col">現在値</th>,
 <th class="align-C" scope="col">前日比</th>,
 <th class="align-C" scope="col">前日比率</th>,
 <th class="align-C" scope="col">更新日時</th>,
 <th>
 <a href="/web/market/data/n225.html" target="_top">日経225</a>
 </th>,
 <th>
 <a href="/web/market/data/topx.html" target="_top">TOPIX</a>
 </th>,
 <th>
 <a href="/web/market/data/jpx400.html" target="_top">JPX日経400</a>
 </th>,
 <th>
 <a href="/web/market/data/tspm.html" target="_top">東証プライム市場指数</a>
 </th>,
 <th>
 <a href="/web/market/data/tssm.html" target="_top">東証スタンダード市場指数</a>
 </th>,
 <th>
 <a href="/web/market/data/tsgm.html" target="_top">東証グロース市場指数</a>
 </th>,
 <th>
 <a href="/web/market/data/ts1ex.html" target="_top">旧東証市場第一部指数</a>
 </th>,
 <th>
 <a href="/web/market/data/tssm20.html" target="_top">東証スタンダード市場TOP20指数</a>
 </th>,
 <th>
 <a href="/web/market/data/tsgmc.html" target="_top">東証グロース市場Core指数</a>
 </th>,
 <th>
 <a href="/web/market/data/mthr.html" target=

In [12]:
titles = []
for elem in elems:
    try:
        title = elem.contents[1].text
        titles.append(title)
    except:
        continue

In [13]:
pickup_links = []
for elem in elems:
    try:
        pickup_links.append("https://www.rakuten-sec.co.jp" + elem.find('a').get('href'))
    except:
        continue

In [14]:
pickup_links2 = []
for url in pickup_links:
    try:
        link = requests.get(url)
        soup = BeautifulSoup(link.content, "html.parser")
        elems = soup.find_all('iframe')
        pickup_links2.append(elems[0].get('src'))
    except:
        continue

In [15]:
link2 = requests.get(pickup_links2[0])
soup = BeautifulSoup(link2.content, "html.parser")
elems3 = soup.find_all('td')

In [16]:
market = []
for url in pickup_links2:
    try:
        link = requests.get(url)
        soup = BeautifulSoup(link.content, "html.parser")
        elems = soup.find_all('td')
        rmove = elems[0].text
        market.append(rmove.replace('\n', "").replace('\r', "").replace('\xa0', "").replace('\t', ""))
    except:
        continue

In [17]:
import pandas as pd
import numpy as np
import os
df = pd.DataFrame(list(zip(titles,market)),columns = ['index', 'rate'])

In [18]:
#先頭および末尾のスペースを削除し、カッコ内の値をカッコもろとも削除
df['rate'] = df['rate'].replace({'\s':'', "\(.*\)":""},regex=True)
# In[72]:
if not os.path.isdir('/tmp/'):
    os.mkdir('/tmp/')
df.to_csv('/tmp/market.csv')

In [27]:
###20240413
#    csvfiles = download()
csvfiles = None

#これは個別実行する際には最終行に移動
#mailjob(csvfiles)

In [20]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.utils import formatdate
import json
import urllib.request
import sys

In [21]:
def create_message(from_addr, to_addr, bcc_addrs, subject, body):
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = from_addr
    msg['To'] = to_addr
    msg['Bcc'] = bcc_addrs
    msg['Date'] = formatdate()
    return msg

In [22]:
def send(from_addr, to_addrs, msg, my_password):
    smtpobj = smtplib.SMTP('smtp.gmail.com', 587) # GmailのSMTPサーバーへ
    smtpobj.ehlo()
    smtpobj.starttls()
    smtpobj.ehlo()
    smtpobj.login(from_addr, my_password)
    smtpobj.sendmail(from_addr, to_addrs, msg.as_string())
    smtpobj.close()

In [32]:
def mailjob(csvfiles):
#    with open('secret.json') as f:
#        info = json.load(f)
        
#    FROM_ADDRESS = info['email']
#    MY_PASSWORD = info['password']
    FROM_ADDRESS = 'ricepyth@gmail.com'
    MY_PASSWORD = 'vlvnfdqsnbkwqxpm'

    
    TO_ADDRESS = 'ricewoods@gmail.com'
    BCC = ''
    SUBJECT = '国内外マーケット指標'
    BODY = '本日の結果になります。'

    msg = create_message(FROM_ADDRESS, TO_ADDRESS, BCC, SUBJECT, BODY)

    with open(r"/tmp/market.csv", "rb") as f:
        attachment = MIMEApplication(f.read())
    attachment.add_header("Content-Disposition", "attachment", filename="market.csv")
    msg.attach(attachment)

    for csvfile in csvfiles:
        with open(r"/tmp/" + csvfile, "rb") as f:
            attachment = MIMEApplication(f.read())
        attachment.add_header("Content-Disposition", "attachment", filename=csvfile)
        msg.attach(attachment)

    send(FROM_ADDRESS, TO_ADDRESS, msg, MY_PASSWORD)


In [24]:
def download():
    url = "https://f.irbank.net/files/0000/"
    csvfiles = ['fy-balance-sheet.csv','fy-cash-flow-statement.csv','fy-profit-and-loss.csv','fy-stock-dividend.csv','qq-yoy-net-sales.csv','qq-yoy-operating-income.csv','qq-yoy-ordinary-income.csv','qq-yoy-profit-loss.csv']
    
    for csvfile in csvfiles:
        data = urllib.request.urlopen(url + csvfile).read()
        with open('/tmp/' + csvfile, mode="wb") as f:
            f.write(data)
    
    return csvfiles

In [33]:
mailjob(csvfiles)